In [ ]:
pip install keras_tuner

     |████████████████████████████████| 133 kB 30.4 MB/s 


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:

import os
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import pickle
import tensorflow as tf
from matplotlib.image import imread
import matplotlib.pyplot as pl
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow import keras
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten, Lambda
from keras.preprocessing.sequence import pad_sequences
import keras_tuner as kt
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
tokenizer = Tokenizer(num_words = 8000, filters = '!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")

print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Tensorflow version: 2.8.0
Keras Version: 2.8.0
GPU is available


In [ ]:
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [ ]:

data_dev = []
data_test = []
data_train = []
file_name_dev = "drive/MyDrive/AMLProject/audioDataDev.pickle"
file_name_test = "drive/MyDrive/AMLProject/audioDataTest.pickle"
file_name_train = "drive/MyDrive/AMLProject/audioDataTrain.pickle"

with (open(file_name_dev, "rb")) as f:
    while True:
        try:
            data_dev.append(pickle.load(f))
        except EOFError:
            break
f.close()

with (open(file_name_test, "rb")) as f:
    while True:
        try:
            data_test.append(pickle.load(f))
        except EOFError:
            break
f.close()

with (open(file_name_train, "rb")) as f:
    while True:
        try:
            data_train.append(pickle.load(f))
        except EOFError:
            break
f.close()

In [ ]:
data_test[1][4]

'0    Hi expo\nName: 4, dtype: object'

In [ ]:
X_dev = []
X_dev_text = []
y_dev = []
X_train = []
X_train_text = []
y_train = []
X_test = []
X_test_text = []
y_test = []

for i in range(len(data_dev)-1):
    X_dev.append(data_dev[i+1][3])
for i in range(len(data_dev)-1):
    X_dev_text.append(data_dev[i+1][4])
for i in range(len(data_dev)-1):
    y_dev.append(data_dev[i+1][6:12])
    
for i in range(len(data_test)-1):
    X_test.append(data_test[i+1][3])
for i in range(len(data_test)-1):
    X_test_text.append(data_test[i+1][4])
for i in range(len(data_test)-1):
    y_test.append(data_test[i+1][6:12])
    
for i in range(len(data_train)-1):
    X_train.append(data_train[i+1][3])
for i in range(len(data_train)-1):
    X_train_text.append(data_train[i+1][4])
for i in range(len(data_train)-1):
    y_train.append(data_train[i+1][6:12])

X_dev = np.array(X_dev)
X_dev_text = np.array(X_dev_text)
y_dev = np.where(np.array(y_dev)> 0.5, 1, 0)
X_test = np.array(X_test)
X_test_text = np.array(X_test_text)
y_test = np.where(np.array(y_test) > 0.5, 1, 0)
X_train = np.array(X_train)
X_train_text = np.array(X_train_text)
y_train = np.where(np.array(y_train) > 0.5, 1, 0)

In [ ]:
y_dev

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 1, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
def sent(X, y):
    data = []
    for i in range(1, len(X)):
        #add text
        temp = X[i].split('\n')
        text_obj = temp[0][6:]

        row = [text_obj, np.array(y[i])]
        data.append(row)

    df = pd.DataFrame(data, columns = ['text', 'sentiment'])
    return df

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def remove_stopwords(text):
    sw = stopwords.words('english')
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    return " ".join(text)

def sequences(dframe):
    tokenizer = Tokenizer(num_words = 8000, filters = '!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
    tokenizer.fit_on_texts(dframe['text'])
    X = tokenizer.texts_to_sequences(dframe['text'].values)
    y = dframe['sentiment'].values
    return X,y

In [ ]:
dev_df = sent(X_dev_text, y_dev)
dev_df['text'] = dev_df['text'].apply(remove_punctuation).apply(remove_stopwords)
dev = sequences(dev_df)

test_df = sent(X_test_text, y_test)
test_df['text'] = test_df['text'].apply(remove_punctuation).apply(remove_stopwords)
test = sequences(test_df)

train_df = sent(X_train_text, y_train)
train_df['text'] = train_df['text'].apply(remove_punctuation).apply(remove_stopwords)
train = sequences(train_df)

In [ ]:
X_dev_text = pad_sequences(dev[0])
y_dev_text = pad_sequences(dev[1])
X_test_text = pad_sequences(test[0])
y_test_text = pad_sequences(test[1])
X_train_text = pad_sequences(train[0])
y_train_text = pad_sequences(train[1])

In [ ]:
X_train_text = X_train_text.reshape(X_train_text.shape[0], 1, X_train_text.shape[1])
X_test_text = X_test_text.reshape(X_test_text.shape[0], 1, X_test_text.shape[1])
X_dev_text = X_dev_text.reshape(X_dev_text.shape[0], 1, X_dev_text.shape[1])

In [ ]:
print(train_df)
print(train)

                                        text           sentiment
0                     ow might wonder buying  [0, 0, 0, 0, 0, 0]
1            multiple copies albums like tha  [1, 0, 0, 0, 0, 0]
2                  ello name fj cava mba fro  [0, 1, 0, 0, 0, 0]
3                 also owner bayview webspot  [0, 1, 0, 0, 0, 0]
4                      oday im going talk ge  [0, 0, 0, 0, 0, 0]
...                                      ...                 ...
16321                            doesnt work  [0, 0, 0, 0, 0, 0]
16322              uhh thing really interest  [0, 0, 0, 0, 0, 0]
16323              set uhh small island coas  [0, 0, 0, 0, 1, 0]
16324  clears throat scenes really well done  [0, 0, 0, 0, 0, 0]
16325                   villages buildings u  [1, 0, 0, 0, 0, 0]

[16326 rows x 2 columns]
([[41, 115, 734, 640], [1646, 5532, 5533, 5, 197], [63, 4, 3674, 5534, 2769, 785], [26, 844, 5535, 5536], [267, 6, 16, 38, 1263], [281, 19, 845, 1898], [24, 243, 641, 458, 30], [5537, 2770, 6, 1

In [ ]:
for i in range(10):
  print(y_train_text[i])

print("+++++++++++++++++++")
for i in range(11):
  print(y_train[i])


[0 0 0 0 0 0]
[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 1 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[1 0 0 0 0 0]
+++++++++++++++++++
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 1 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[1 0 0 0 0 0]


In [ ]:
X_train.shape

(16327, 128, 850)

In [ ]:
X_train_audio = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2],1)
X_test_audio = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
X_dev_audio = X_dev.reshape(X_dev.shape[0], X_dev.shape[1], X_dev.shape[2], 1)

In [ ]:
X_train.shape

(16327, 128, 850)

In [ ]:
def get_2d_conv_model():
    model = Sequential()
    model.add(Convolution2D(1, 3, activation='relu', input_shape=(128, 850,1)))
    model.add(MaxPool2D(pool_size=(2)))
    model.add(Dropout(0.25))
    model.add(Convolution2D(1, 3, activation='relu'))
    model.add(MaxPool2D(pool_size=(2)))
    model.add(Dropout(0.25))
    model.add(Convolution2D(1, 1, activation='relu'))
    model.add(MaxPool2D(pool_size=(2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(6, activation='sigmoid'))
    return model

In [ ]:
from keras import losses, models, optimizers
from keras.activations import relu, softmax
from keras.layers import Dense, Dropout, Input, Convolution2D, BatchNormalization, Activation, MaxPool2D, Flatten
model_cnn = get_2d_conv_model()

In [ ]:
extractor = keras.Model(inputs=model_cnn.inputs,
                        outputs=[layer.output for layer in model_cnn.layers])
features = extractor(X_train_audio)


In [ ]:
print(len(features))
print(len(features[0]))
print(len(features[0][0]))
print(len(features[0][0][0]))
print(len(features[0][0][0][0]))

11
16327
126
848
1


In [ ]:
audio_features =  features

In [ ]:
X_train_text.shape

(16326, 1, 9)

In [ ]:
def get_model_lstm_image():
    lstm = keras.Sequential()
    lstm.add(LSTM(32, input_shape=(128, 850), activation='relu', return_sequences=False))
    lstm.add(Dense(6, activation='sigmoid'))
    return lstm

In [ ]:
model_lstm_image = get_model_lstm_image()
model_lstm_image.compile(loss='binary_crossentropy',metrics='BinaryAccuracy', optimizer=adam)

In [ ]:
model_lstm_image.fit(X_train, y_train, epochs=5, batch_size=10, verbose=1, shuffle=True)

Epoch 1/5
1633/1633 [==============================] - 80s 49ms/step - loss: nan - binary_accuracy: 0.8710
Epoch 2/5
1633/1633 [==============================] - 79s 48ms/step - loss: nan - binary_accuracy: 0.8710
Epoch 3/5
1633/1633 [==============================] - 79s 48ms/step - loss: nan - binary_accuracy: 0.8710
Epoch 4/5
1633/1633 [==============================] - 78s 48ms/step - loss: nan - binary_accuracy: 0.8710
Epoch 5/5
1633/1633 [==============================] - 78s 48ms/step - loss: nan - binary_accuracy: 0.8710


In [ ]:
adam = Adam(0.0001)
model_cnn.compile(optimizer=adam,loss='binary_crossentropy',metrics=['AUC','BinaryAccuracy'])
model_cnn.fit(X_train, y_train, epochs=30, validation_data=(X_dev, y_dev), batch_size=10)

Epoch 1/30
1633/1633 [==============================] - 163s 99ms/step - loss: 0.8695 - auc: 0.6248 - binary_accuracy: 0.8007 - val_loss: 0.3516 - val_auc: 0.7068 - val_binary_accuracy: 0.8725
Epoch 2/30
1633/1633 [==============================] - 150s 92ms/step - loss: 0.3947 - auc: 0.6932 - binary_accuracy: 0.8562 - val_loss: 0.3568 - val_auc: 0.7294 - val_binary_accuracy: 0.8802
Epoch 3/30
1633/1633 [==============================] - 151s 93ms/step - loss: 0.3552 - auc: 0.7224 - binary_accuracy: 0.8664 - val_loss: 0.3574 - val_auc: 0.7432 - val_binary_accuracy: 0.8820
Epoch 4/30
1633/1633 [==============================] - 157s 96ms/step - loss: 0.3443 - auc: 0.7375 - binary_accuracy: 0.8696 - val_loss: 0.3456 - val_auc: 0.7489 - val_binary_accuracy: 0.8821
Epoch 5/30
1633/1633 [==============================] - 151s 92ms/step - loss: 0.3405 - auc: 0.7441 - binary_accuracy: 0.8701 - val_loss: 0.3433 - val_auc: 0.7530 - val_binary_accuracy: 0.8822
Epoch 6/30
1633/1633 [=============

In [ ]:
yhat_imge = model_cnn.predict(X_test)

In [ ]:
yhat_imge_train = model_cnn.predict(X_train)

In [ ]:

yhat_imge_train = np.where(yhat_imge_train > 0.5, 1, 0)
results=[]
total = 0
similar = 0
ones = 0
truth = 0
for i in range(len(y_train)):
    for j in range(len(y_train[i])):
        if y_train[i][j] == yhat_imge_train[i][j]:
            similar += 1
        if yhat_imge_train[i][j] == 1:
            ones += 1
        if y_train[i][j] == 1:
            truth += 1
        total += 1
print(truth)
print(ones)
acc_image = similar/total

print('>%.3f' % acc_image)
results.append(acc_image)

12637
610
>0.873


In [ ]:
print(yhat_image)

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [ ]:
def get_model_lstm():
    lstm = keras.Sequential()
    lstm.add(LSTM(32, input_shape = (1, X_train_text.shape[2]), activation='relu', return_sequences=False))
    lstm.add(Dense(6, activation='sigmoid'))
    return lstm

adam = Adam(0.0001)
model_lstm = get_model_lstm()
model_lstm.compile(loss='binary_crossentropy',metrics='BinaryAccuracy', optimizer=adam)

In [ ]:
model_lstm.fit(X_train_text, y_train_text, epochs=30, batch_size=10, verbose=1, shuffle=True)

Epoch 1/30
1633/1633 [==============================] - 4s 2ms/step - loss: 19.0647 - binary_accuracy: 0.7578
Epoch 2/30
1633/1633 [==============================] - 3s 2ms/step - loss: 8.2643 - binary_accuracy: 0.8049
Epoch 3/30
1633/1633 [==============================] - 3s 2ms/step - loss: 4.5875 - binary_accuracy: 0.8119
Epoch 4/30
1633/1633 [==============================] - 3s 2ms/step - loss: 2.7550 - binary_accuracy: 0.8195
Epoch 5/30
1633/1633 [==============================] - 3s 2ms/step - loss: 1.7769 - binary_accuracy: 0.8294
Epoch 6/30
1633/1633 [==============================] - 3s 2ms/step - loss: 1.2318 - binary_accuracy: 0.8371
Epoch 7/30
1633/1633 [==============================] - 4s 2ms/step - loss: 0.8675 - binary_accuracy: 0.8502
Epoch 8/30
1633/1633 [==============================] - 3s 2ms/step - loss: 0.6798 - binary_accuracy: 0.8571
Epoch 9/30
1633/1633 [==============================] - 3s 2ms/step - loss: 0.5706 - binary_accuracy: 0.8622
Epoch 10/30
1633/1

In [ ]:
model_lstm.fit(X_train_text, y_train_text, epochs=10, batch_size=10, verbose=1, shuffle=True)

Epoch 1/10
1633/1633 [==============================] - 4s 2ms/step - loss: 0.3546 - binary_accuracy: 0.8698
Epoch 2/10
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3538 - binary_accuracy: 0.8698
Epoch 3/10
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3525 - binary_accuracy: 0.8701
Epoch 4/10
1633/1633 [==============================] - 4s 2ms/step - loss: 0.3530 - binary_accuracy: 0.8699
Epoch 5/10
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3515 - binary_accuracy: 0.8698
Epoch 6/10
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3515 - binary_accuracy: 0.8698
Epoch 7/10
1633/1633 [==============================] - 4s 2ms/step - loss: 0.3507 - binary_accuracy: 0.8699
Epoch 8/10
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3498 - binary_accuracy: 0.8700
Epoch 9/10
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3491 - binary_accuracy: 0.8699
Epoch 10/10
1633/16

In [ ]:
yhat_text_train = model_lstm.predict(X_train_text)

In [ ]:
yhat_text_train = np.where(yhat_text_train > 0.5, 1, 0)
results=[]
total = 0
similar = 0
ones = 0
truth = 0
for i in range(len(y_train_text)):
    for j in range(len(y_train_text[i])):
        if y_train_text[i][j] == yhat_text_train[i][j]:
            similar += 1
        if yhat_text_train[i][j] == 1:
            ones += 1
        if y_train_text[i][j] == 1:
            truth += 1
        total += 1
print(truth)
print(ones)
acc_image = similar/total

print('>%.3f' % acc_image)
results.append(acc_image)

12637
579
>0.869


In [ ]:
y_image_after = yhat_imge_train[1:]


In [ ]:
print(len(y_image_after))
print(len(yhat_text_train))
print(len(y_train_text))

16326
16326
16326


In [ ]:
concate = np.concatenate((y_image_after, yhat_text_train), axis = 1)

In [ ]:
concate.shape

(16326, 12)

In [ ]:
def get_model_lstm_final():
    lstm = keras.Sequential()
    lstm.add(LSTM(32, input_shape = (1, 12), activation='relu', return_sequences=False))
    lstm.add(Dense(6, activation='sigmoid'))
    return lstm

In [ ]:
concate = concate.reshape(concate.shape[0], 1, concate.shape[1])
model_lstm_final = get_model_lstm_final()
model_lstm_final.compile(loss='binary_crossentropy',metrics='BinaryAccuracy', optimizer=adam)
model_lstm_final.fit(concate, y_train_text, epochs=30, batch_size=10, verbose=1, shuffle=True)

Epoch 1/30
1633/1633 [==============================] - 4s 2ms/step - loss: 0.5396 - binary_accuracy: 0.8709
Epoch 2/30
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3731 - binary_accuracy: 0.8710
Epoch 3/30
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3433 - binary_accuracy: 0.8710
Epoch 4/30
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3392 - binary_accuracy: 0.8710
Epoch 5/30
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3383 - binary_accuracy: 0.8710
Epoch 6/30
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3379 - binary_accuracy: 0.8710
Epoch 7/30
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3376 - binary_accuracy: 0.8710
Epoch 8/30
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3374 - binary_accuracy: 0.8710
Epoch 9/30
1633/1633 [==============================] - 3s 2ms/step - loss: 0.3373 - binary_accuracy: 0.8710
Epoch 10/30
1633/16

In [ ]:
yhat_final = model_lstm_final.predict(concate)

In [ ]:
yhat_final = np.where(yhat_final > 0.5, 1, 0)
results=[]
total = 0
similar = 0
ones = 0
truth = 0
for i in range(len(y_train_text)):
    for j in range(len(y_train_text[i])):
        if y_train_text[i][j] == yhat_final[i][j]:
            similar += 1
        if yhat_final[i][j] == 1:
            ones += 1
        if y_train_text[i][j] == 1:
            truth += 1
        total += 1
print(truth)
print(ones)
acc_image = similar/total

print('>%.3f' % acc_image)
results.append(acc_image)

12637
609
>0.873


In [ ]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 848, 1)       10        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 424, 1)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 424, 1)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 422, 1)        10        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 211, 1)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 30, 211, 1)        0

In [ ]:
model_lstm.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_lstm_final.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 32)                5760      
                                                                 
 dense_6 (Dense)             (None, 6)                 198       
                                                                 
Total params: 5,958
Trainable params: 5,958
Non-trainable params: 0
_________________________________________________________________


In [ ]:
yhat_imge_test = model_cnn.predict(X_test)
yhat_imge_test = np.where(yhat_imge_test > 0.5, 1, 0)

yhat_text_image = model_lstm.predict(X_test_text)
yhat_text_image = np.where(yhat_text_image > 0.5, 1, 0)

In [ ]:
print(len(yhat_imge_test))
print(len(yhat_text_image))
y_image_drop = yhat_imge_test[1:]
print(len(y_image_drop))

4662
4661
4661


In [ ]:
concate_final = np.concatenate((y_image_drop, yhat_text_image), axis = 1)
concate_final = concate_final.reshape(concate_final.shape[0], 1, concate_final.shape[1])

yhat_final_test = model_lstm_final.predict(concate)

In [ ]:
yhat_final_test = np.where(yhat_final_test > 0.5, 1, 0)
results=[]
total = 0
similar = 0
ones = 0
truth = 0
for i in range(len(y_test_text)):
    for j in range(len(y_test_text[i])):
        if y_test_text[i][j] == yhat_final_test[i][j]:
            similar += 1
        if yhat_final_test[i][j] == 1:
            ones += 1
        if y_test_text[i][j] == 1:
            truth += 1
        total += 1
print(truth)
print(ones)
acc_image = similar/total

print('>%.3f' % acc_image)
results.append(acc_image)

3567
149
>0.870


In [ ]:
correctness = []
for j in range(len(y_test_text[i])):
    correct = 0
    total = 0
    for i in range(len(y_test_text)):
        if y_test_text[i][j] == yhat_final_test[i][j]:
          correct += 1
        total += 1
    correctness.append(correct/total)

In [ ]:
correctness

[0.6429950654366016,
 0.8759922763355503,
 0.8586140313237502,
 0.9729671744260888,
 0.9092469427161554,
 0.9630980476292641]

In [ ]:
total = 0
for i in correctness:
  total += i
ave = total/6
print(ave)

0.8704855896445686


In [ ]:

model = get_2d_conv_model()


NameError: ignored